## X-ray time lag explorer

This notebook uses the pyLag X-ray spectral timing package to interactively explore time lags as a function of Fourier frequency, and can be used to look for X-ray reverberation signals from the inner accretion disc.

Requires the pylag and ipympl packages to be installed (in addition to their dependencies).

D.R. Wilkins (May 2024)

In [2]:
%matplotlib inline
from pylag import *
from ipywidgets import interactive, FloatSlider, IntSlider
from matplotlib.patches import Rectangle

### Lag-frequency spectrum

First we explore the lag vs. frequency spectrum to understand how the time lags vary as a function of Fourier frequency.

We will compute the lag-frequency spectrum between the reflection-dominated soft X-ray band (0.3-1keV) and the continuum-dominated hard X-ray band.

We load a list of light curves in each energy band where each light curve in the list is the light curve obtained from one single observation. The lag-frequency spectrum will be averaged over all of the observations. The get_lclist function can either take the name of a single FITS file containing a light curve, or we can specify a 'glob' (the * notation to select multiple filenames in the normal way).

We also need to specify the frequency bins for the spectrum. We will start with 10 bins, but then later on we will have the option to adjust the bins interactively.

In the lag-frequency spectrum, by convention, a positive lag corresponds to variations in the hard X-ray band lagging behind those in the soft X-ray band. In this case, reverberation corresponds to the soft X-ray band (which is most strongly dominated by the reflection) lagging begind the hard band (dominated by the continuum), so reverberation from the disc is detected over ranges of Fourier frequency in which the lag is negative.

The light curves may have gaps due to the filtering of background flares (particularly for XMM-Newton data). If there are gaps in the light curves we will need to specify interp_gaps=True to interpolate over them. Alternatively, we can use light curve files in which time-based filtering for background flares has not been performed.

In [4]:
lc_hard = get_lclist('lightcurves/for_lagfreq/*tbin10_en1200-4000.lc_corr', interp_gaps=True)
lc_soft = get_lclist('lightcurves/for_lagfreq/*tbin10_en300-1000.lc_corr', interp_gaps=True)
lf = LagFrequencySpectrum(10, lc_hard, lc_soft)

/home/drw/python/pylag/pylag/lightcurve.py:373: RuntimeWarning: invalid value encountered in sqrt
  new_error[gap_start:gap_end] = np.sqrt(new_rate[gap_start:gap_end] / self.dt)
/home/drw/python/pylag/pylag/coherence.py:176: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt((1 - coh) / (2 * coh * num_freq))


In [5]:
def interact_lagfreq(fmin, fmax, Nf):
    lf.bins = LogBinning(10**fmin, 10**fmax, Nf)
    p = Plot(lf)
interactive_lagfreq = interactive(interact_lagfreq, fmin=FloatSlider(min=-6, max=-2, step=0.1, value=-5), fmax=FloatSlider(min=-6, max=-2, step=0.1, value=-2), Nf=IntSlider(min=1, max=50, step=1, value=20))
output_lf = interactive_lagfreq.children[-1]
output_lf.layout.height = '500px'

In [6]:
interactive_lagfreq

interactive(children=(FloatSlider(value=-5.0, description='fmin', max=-2.0, min=-6.0), FloatSlider(value=-2.0,…

### Lag-energy spectrum

Now that we have an understanding of how the time lags evolve with frequency, we can explore the energy-dependence of the time lags (i.e. the relative times at which corresponding variations are seen at different X-ray energies) using the lag-energy spectrum in a specfic range of frequencies.

We will need to load a list of light curves in each energy band. This is handled by the EnergyLCList object, which reads a collection of FITS light curve files for different energy bands, collected over a number of observations (each individual file should be the light curve in a specific energy band from one observation). The energy corresponding to each light curve is read from the filename (see the pyLag documentation for details of how the files should be named). We also specify the frequency range over which to calculate the lag-energy spectrum. We will change this interactively, but we just need to specify some starting values to get everything set up.

As above, the light curves may have gaps due to the filtering of background flares (particularly for XMM-Newton data). If there are gaps in the light curves we will need to specify interp_gaps=True to interpolate over them. Alternatively, we can use light curve files in which time-based filtering for background flares has not been performed.

The lag-energy spectrum will be plotted next to the lag-ferquency spectrum from above (use the above controls to change the binning of the lag-frequency spectrum). Use the sliders to adjust the frequency range over which the lag-energy spectrum is computed (they correspond to the log10 of the centre of the frequency range, and the width of the range in dex).

If you select a range in which the lag-frequency plot is positive, you should see the hard X-ray lag (attributed to the propgation of luminosity fluctuations through the corona). If you select a range in which the lag-frequency spectrum turns negative, you may see reverberation from the accretion disc. In this case, the lag-energy spectrum should look like the reflection spectrum and should show the soft excess as well as the broad iron K line lagging behind the continuum-dominated bands.

In [7]:
fmin, fmax = 3e-4, 1e-3
lcl = EnergyLCList('lightcurves/for_lagen/0890670201*.lc_corr', interp_gaps=True)
le = LagEnergySpectrum(fmin, fmax, lcl)

/usr/lib64/python3.12/site-packages/numpy/lib/function_base.py:1599: RuntimeWarning: invalid value encountered in cast
  return interp_func(x, xp, fp, left, right)


In [8]:
def interact_lagfreq_lagen(fcent, fwidth):
    fmin, fmax = 10**(fcent - fwidth/2), 10**(fcent + fwidth/2)
    le.freq_range = (fmin, fmax)
    p = MultiPlot([lf, le], cols=True, sharex=False, wspace=0.25, figsize=(14,4))
    ylim = p[0]._ax.get_ylim()
    p[0]._ax.add_patch(Rectangle((fmin, ylim[0]), fmax-fmin, ylim[1]-ylim[0], facecolor='C0', alpha=0.25))
    
interactive_lagfreq_lagen = interactive(interact_lagfreq_lagen, fcent=FloatSlider(min=-6, max=-2, step=0.01, value=-4), fwidth=FloatSlider(min=0.1, max=1, step=0.01, value=0.5))
output_lf_le = interactive_lagfreq_lagen.children[-1]
output_lf_le.layout.height = '450px'

In [9]:
interactive_lagfreq_lagen

interactive(children=(FloatSlider(value=-4.0, description='fcent', max=-2.0, min=-6.0, step=0.01), FloatSlider…